In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np

# Google developer API key
from api_keys import gkey_jr

In [6]:
# read addresses file and create dataframe
addresses_file = "../data/cleandata/County_Adds.csv"
addresses_df = pd.read_csv(addresses_file, encoding="ISO-8859-1")
addresses_df.set_index(addresses_df.iloc[:,0],inplace=True)
addresses_df.drop(columns='Unnamed: 0',inplace=True)
del addresses_df.index.name
addresses_df.head()

,zip,address,lat,lng,County
0,78944,"187 BUENGER RD, INDUSTRY, TX 78944",29.967902,-96.493013,Austin
1,78944,"7523 ERNST PKWY, INDUSTRY, TX 78944",29.968208,-96.501229,Austin
2,78944,"7586 ABKE LN, INDUSTRY, TX 78944",29.968105,-96.499605,Austin
3,78944,"7474 ERNST PKWY, INDUSTRY, TX 78944",29.968003,-96.500844,Austin
4,78944,"7571 ABKE LN, INDUSTRY, TX 78944",29.967776,-96.499775,Austin


In [7]:
#find max and min lats and lngs in Houston
max_lat=addresses_df['lat'].max()
min_lat=addresses_df['lat'].min()
max_lng=addresses_df['lng'].max()
min_lng=addresses_df['lng'].min()
houston_lng_range=max_lng-min_lng
houston_lat_range=max_lat-min_lat

print(f"Houston max lat is: {max_lat}.")
print(f"Houston min lat is: {min_lat}.")
print(f"Houston max lng is: {max_lng}.")
print(f"Houston min lng is: {min_lng}.")

#find increment for walking distance (miles for ranges found in google maps)
miles_per_lat=houston_lat_range/150
miles_per_lng=houston_lng_range/160

Houston max lat is: 30.655781.
Houston min lat is: 28.870009000000003.
Houston max lng is: -94.3831.
Houston min lng is: -96.594711.


In [10]:
#create arrays for lat and lng for API calls from google places (each call will be for 1 mile increments)
lat_range=np.arange(min_lat,max_lat,miles_per_lat)
lng_range=np.arange(min_lng,max_lng,miles_per_lng)
lat_lngs=[]
for lat in lat_range:
    for lng in lng_range:
        lat_lngs.append((lat,lng))
        
f"{lat_lngs[10000][0]}, {lat_lngs[10000][1]}"

'29.60812809333335, -95.48890549999992'

In [11]:
ammenities_df = pd.DataFrame(data={"lat":"", "lng":"","# Restaurants":"" }, index = [i for i in range(len(lat_lngs))])
ammenities_df

,lat,lng,# Restaurants
0,,,
1,,,
2,,,
3,,,
4,,,
5,,,
6,,,
7,,,
8,,,
9,,,


In [8]:
# base url and query specifics
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 1610
target_type = "bar"

# geocoordinates
#for i in range(24000,len(lat_lngs),1):
target_coordinates = "29.71527441,-95.43361522"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey_jr
}

# run a request using our params dictionary
response = requests.get(base_url, params=params)
    
#count number of ammenities
places_data = response.json()
    
#count number fo ammenities in first page
places=places_data["results"]
names = [place["name"] for place in places]
ammenity_count_per_page=len(names)
ammenity_count=len(names)
    
#see if more than one page of results exists
while ammenity_count_per_page==20:
    ammenity_count_per_page=0
    if "next_page_token" in places_data:
        
        page_token=places_data["next_page_token"]
        
        params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey_jr,
        "pagetoken":page_token
        }  

        response = requests.get(base_url, params=params)
        places_data = response.json()
        places=places_data["results"]
        names = [place["name"] for place in places]
        ammenity_count_per_page=len(names)
        ammenity_count=ammenity_count+ammenity_count_per_page
    
#populate dataframe
#ammenities_df['lat'].iloc[i]=lat_lngs[i][0]
#ammenities_df['lng'].iloc[i]=lat_lngs[i][1]
#ammenities_df['# Restaurants'].iloc[i]=ammenity_count
#ammenities_df
ammenity_count

9

In [34]:
ammenities_df

,lat,lng,# Restaurants
0,28.87,-96.5947,0
1,28.87,-96.5809,0
2,28.87,-96.5671,0
3,28.87,-96.5532,0
4,28.87,-96.5394,0
5,28.87,-96.5256,0
6,28.87,-96.5118,0
7,28.87,-96.498,0
8,28.87,-96.4841,0
9,28.87,-96.4703,0


In [35]:
#export data to csv
output_data_file = "../data/rawdata/restaurant_count.csv"
ammenities_df.to_csv(output_data_file, index=False, header=True)

In [22]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "error_message": "You have exceeded your daily request quota for this API. If you did not set a custom daily request quota, verify your project has an active billing account: http://g.co/dev/maps-no-account",
    "html_attributions": [],
    "results": [],
    "status": "OVER_QUERY_LIMIT"
}


In [129]:
places=places_data["results"]
places
names = [place["name"] for place in places]
ammenity_count=len(names)
ammenity_count

10